# 6. フレームワーク Scrapy

## 6-1. Scrapyの概要

- Webページのリンクの抽出
- robots.txtの取得と拒否されているページのクロール防止
- XMLサイトマップの取得とリンクの抽出
- ドメインごと・IPアドレスごとのダウンロード間隔の調整
- 複数のクロール先の並列処理
- 重複するURLのクロール防止
- エラー時の回数制限付きのリトライ
- クローラーのジョブの管理

&nbsp;

- [Twisted](https://twistedmatrix.com/)
- [Scrapy](https://scrapy.org)

### 6-1-1. Scrapyのインストール

### 6-1-2. Spiderの実行

- Spider（class）
  - 対象のWebサイトごとにSpiderを作成
  - クローリングの設定
  - スクレイピングの処理

&nbsp;

- .jlファイル
  - JSON Lines形式
  - 各行にJSONオブジェクトを持つテキスト形式で、ファイルの追記が容易

In [2]:
!scrapy runspider my_spider.py -o items.jl

2023-05-21 22:23:24 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-05-21 22:23:24 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-05-21 22:23:24 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2023-05-21 22:23:24 [py.warnings] WARNING: /Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/venv/lib/python3.10/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatib

## 6-2 Spiderの作成と実行